In [1]:
import pandas as pd
import numpy as np

combined = pd.read_csv('Input Data/clean_data.csv')
combined.head()

C:\Users\Adam\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,date_time,date_x,time,loc_t ype,zip_code,council_district,apd_sector,apd_district,latitude,longtitude,...,zip78749,zip78750,zip78751,zip78752,zip78753,zip78754,zip78756,zip78757,zip78758,zip78759
0,2014-10-01 20:19:00,10/01/2014,2019.0,RESIDENCE / HOME,78723.0,1.0,ID,1,30.315506,-97.690678,...,0,0,0,0,0,0,0,0,0,0
1,2014-10-05 16:33:00,10/05/2014,1633.0,RESIDENCE / HOME,78723.0,1.0,ID,1,30.315506,-97.690678,...,0,0,0,0,0,0,0,0,0,0
2,2012-01-20 22:30:00,01/20/2012,2230.0,PARKING LOTS / GARAGE,78723.0,1.0,ID,1,30.320667,-97.687671,...,0,0,0,0,0,0,0,0,0,0
3,2011-01-01 20:00:00,01/01/2011,2000.0,RESIDENCE / HOME,78723.0,1.0,ID,1,30.320667,-97.687671,...,0,0,0,0,0,0,0,0,0,0
4,2012-03-06 05:00:00,03/06/2012,500.0,RESIDENCE / HOME,78723.0,1.0,ID,1,30.320667,-97.687671,...,0,0,0,0,0,0,0,0,0,0


In [2]:
dropped_columns = ['date_time', 'date_x', 'time', 'loc_t ype', 'zip_code', 'council_district', 'apd_sector', 'apd_district']
dropped_columns += ['latitude', 'longtitude', 'location', 'year']
combined = combined.drop(dropped_columns, axis = 1)
len(combined.columns)

183

In [3]:
from sklearn.model_selection import train_test_split

#drop rows that are not complete
combined = combined.dropna(subset= combined.columns)

cols_to_use = ["temp_max", "temp_min", "prec_avg", 'new_year', 'mlk_day', 'wash_bday', 'mem_day', 'ind_day', 
      'labor_day', 'col_day', 'vet_day', 'thanksgiving', 'christmas', 'P0030001','P0030002_pct', 'P0030003_pct', 'P0030004_pct', 'P0030005_pct', 'P0030006_pct', 
      'P0040003_pct', 'P0130001', 'B19113_001E', 'B17001_001E_pct', 'C18120_003E_pct']

Y, X = combined.Target1, combined[cols_to_use]
#Y, X = combined.Target1, combined.drop(['Target1', 'Target2'], axis = 1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 3, stratify = Y)


In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics
from sklearn.metrics import roc_curve, auc

# Logistic Regression
clf = LogisticRegression().fit(X_train, Y_train)

fpr, tpr, threshold = metrics.roc_curve(Y_test, clf.predict_proba(X_test)[:,1])
logAUC = metrics.roc_auc_score(Y_test, clf.predict_proba(X_test)[:,1])

print('The logistic regression accuracy is: ' + str(clf.score(X_test, Y_test)))
print('The logistic regression AUC is: ' + str(auc(fpr, tpr)))

The logistic regression accuracy is: 0.9871260629467764
The logistic regression AUC is: 0.615703465686215


In [5]:
print(Y_train.value_counts()/len(Y_train))
print(Y_test.value_counts()/len(Y_test))

0.0    0.987127
1.0    0.012873
Name: Target1, dtype: float64
0.0    0.987126
1.0    0.012874
Name: Target1, dtype: float64


In [6]:
from sklearn.model_selection import StratifiedKFold

# Set up stratified k fold split
skf = StratifiedKFold(n_splits=5, random_state=400)
skf.get_n_splits(X_train, Y_train)

5

In [7]:
#obtain AUC for each of the k folds
#auc_lr = []
#for train_index, test_index in skf.split(X_train, Y_train):
    
#    X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
#    Y_train_fold, Y_test_fold = Y_train.iloc[train_index], Y_train.iloc[test_index]
    
#    model = LogisticRegression(C=1e30).fit(X_train_fold, Y_train_fold)
    
#    Y_pred_fold = model.predict_proba(X_test_fold)[:,1]
#    fpr, tpr, thresholds = roc_curve(Y_test_fold, Y_pred_fold)
#    auc_lr.append(auc(fpr, tpr))
    
#print(auc_lr)

In [8]:
from sklearn.svm import SVC
from sklearn.svm import LinearSVC

auc_svm = []
for train_index, test_index in skf.split(X_train, Y_train):
    
    X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    Y_train_fold, Y_test_fold = Y_train.iloc[train_index], Y_train.iloc[test_index]
    
    #clfsvm = SVC(gamma='auto', kernel = "linear", probability = True).fit(X_train_fold, Y_train_fold)
    clfsvm = LinearSVC().fit(X_train_fold, Y_train_fold)
    
    Y_pred_fold = clfsvm.predict(X_test_fold)
    print(clfsvm.score(X_test_fold, Y_test_fold))
    fpr, tpr, thresholds = roc_curve(Y_test_fold, Y_pred_fold)
    auc_svm.append(auc(fpr, tpr))
    print(auc(fpr,tpr))
    
print(auc_svm)




0.9871261252540896
0.5
0.9871261252540896
0.5
0.9871260473698539
0.5
0.9871260473698539
0.5
0.9871320193110458
0.5
[0.5, 0.5, 0.5, 0.5, 0.5]


In [9]:
#print(clfsvm.score(X_test_fold, Y_test_fold))

0.9871320193110458


In [10]:
#from sklearn.neighbors import KNeighborsClassifier
#clfKNN = KNeighborsClassifier(n_neighbors=5).fit(X_train, Y_train)

#Y_pred3 = clfKNN.predict_proba(X_test)[:,1]
#fpr3, tpr3, thresholds3 = roc_curve(Y_test, Y_pred3)
#print(auc(fpr3,tpr3))